<h1><b>Machine Learning in CyberSecurity</b></h1>
<h2>by <b>Prof. Dr. Mario Frtiz</b></h2>
<h2><b>Winter 2020/21</b></h2>
<br/>
<h3><b>Final Project:</b> MI Attack: More Independent, Meaningfully Realitic</h3>

<h3><b>Team:</b> <span style="color:red;">DeMons</span></h3>
<br/>
<h3><b>Members:</b></h3> 
<p><b>Name:</b> Rayhanul Islam Rumel <br/>
<b>Email:</b> s8rarume@stud.uni-saarland.de <br/>
<b>Matriculation Number:</b> 2576541</p>

In [1]:
#Import Libraries

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import Subset
from torch.utils.data import Subset, Dataset, DataLoader
from torchvision.datasets import CIFAR100
import matplotlib.pyplot as plt
from PIL import Image
import time
import os
import copy
import pandas as pd
import random

plt.ion()   # interactive mode
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
def get_target_label_idx(labels, targets, shots=5, test=False):
    """
    Get the indices of labels that are included in targets.
    :param labels: array of labels
    :param targets: list/tuple of target labels
    :return: list with indices of target labels
    """
    final_list = []
    
    for t in targets:
        if test:
            final_list += np.argwhere(np.isin(labels, t)).flatten().tolist()
        else:
            final_list += np.argwhere(np.isin(labels, t)).flatten().tolist()
    
    return final_list

In [3]:
class MyCIFAR100(torchvision.datasets.CIFAR100):
    """Torchvision CIFAR100 class with patch of __getitem__ method to also return the index of a data sample."""

    def __init__(self, *args, **kwargs):
        super(MyCIFAR100, self).__init__(*args, **kwargs)

    def __getitem__(self, index):
        """Override the original method of the CIFAR100 class.
        Args:
            index (int): Index
        Returns:
            triple: (image, target, index) where target is index of the target class.
        """
        if self.train:
            img, target = self.data[index], self.targets[index]
        else:
            img, target = self.test_data[index], self.targets[index]

        # doing this so that it is consistent with all other datasets
        # to return a PIL Image
        # print(type(img))
        img = Image.fromarray(img)


        if self.transform is not None:
            img = self.transform(img)

        if self.target_transform is not None:
            target = self.target_transform(target)

        return img, target, index  # only line changed

In [4]:
#Data Process

target_transform = transforms.Lambda(lambda x: convert_label(x))
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize(mean=[n/255.
                                for n in [129.3, 124.1, 112.4]], std=[n/255. for n in [68.2,  65.4,  70.4]])])
        
train_set = MyCIFAR100(root='data/', train=True, download=True,
                            transform=transform) 

Extracting data/cifar-100-python.tar.gz to data/


In [5]:
normal_class=[0,1,2,3,4,5,6,7,8,9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99]
normal_classes = tuple(normal_class)

In [6]:
train_index = get_target_label_idx(train_set.targets, normal_classes, shots=5)
random.shuffle(train_index) #shuffling before split


In [7]:
# splitting to target and shadowset
train_index_half_len = int(len(train_index)/2)
shadow_set = Subset(train_set, train_index[0:train_index_half_len])
target_set = Subset(train_set, train_index[train_index_half_len:])

shadow_half_len = int(len(shadow_set)/2)
print("shadow half len: ", shadow_half_len)
shadow_train = Subset(shadow_set, list(range(0, shadow_half_len)))
shadow_test = Subset (shadow_set, list(range(shadow_half_len, len(shadow_set))))


target_half_len = int(len(target_set)/2)
print("Target half len: ", target_half_len)
target_train = Subset(target_set, list(range(0, target_half_len)))
target_unknown = Subset(target_set, list(range(target_half_len, len(target_set))))

classes = ('beaver', 'dolphin', 'otter', 'seal', 'whale', 'aquarium fish', 'flatfish', 'ray', 'shark', 'trout',  'orchids', 'poppies', 'roses', 'sunflowers', 'tulips',  'bottles', 'bowls', 'cans', 'cups', 'plates', 
'apples', 'mushrooms', 'oranges', 'pears', 'sweet peppers',  'clock', 'computer keyboard', 'lamp', 'telephone', 'television',  'bed', 'chair', 'couch', 'table', 'wardrobe',  'bee', 'beetle', 'butterfly', 'caterpillar', 'cockroach', 
'bear', 'leopard', 'lion', 'tiger', 'wolf',  'bridge', 'castle', 'house', 'road', 'skyscraper',  'cloud', 'forest', 'mountain', 'plain', 'sea',  'camel', 'cattle', 'chimpanzee', 'elephant', 'kangaroo', 
'fox', 'porcupine', 'possum', 'raccoon', 'skunk',  'crab', 'lobster', 'snail', 'spider', 'worm',  'baby', 'boy', 'girl', 'man', 'woman',  'crocodile', 'dinosaur', 'lizard', 'snake', 'turtle', 
'hamster', 'mouse', 'rabbit', 'shrew', 'squirrel',  'maple', 'oak', 'palm', 'pine', 'willow',  'bicycle', 'bus', 'motorcycle', 'pickup truck', 'train',  'lawn-mower', 'rocket', 'streetcar', 'tank', 'tractor')

shadow half len:  12500
Target half len:  12500


In [8]:
shadow_train_loader = DataLoader(shadow_train, batch_size=64, shuffle=True, num_workers=0)
shadow_test_loader = DataLoader(shadow_test, batch_size=64, shuffle=True, num_workers=0)

In [9]:
""" Shadow Model """

class Shadow_LeNet(nn.Module):
    def __init__(self):
        super(Shadow_LeNet, self).__init__()
        self.conv1 = nn.Conv2d(3,6,5)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(6,16,5)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,100)

    def forward(self, x):
        out = self.pool(F.relu(self.conv1(x)))
        out = self.pool(F.relu(self.conv2(out)))
        out = out.view(out.size(0),-1)
        out = F.relu(self.fc1(out))
        out = F.relu(self.fc2(out))
        out = self.fc3(out)
        return out

In [10]:
""" Base Shadow Model """

class Base_Shadow_LeNet(nn.Module):
    def __init__(self):
        super(Base_Shadow_LeNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1   = nn.Linear(16*5*5, 120)
        self.fc2   = nn.Linear(120, 84)
        self.fc3   = nn.Linear(84, 100)

    def forward(self, x):
        out = F.relu(self.conv1(x))
        out = F.max_pool2d(out, 2)
        out = F.relu(self.conv2(out))
        out = F.max_pool2d(out, 2)
        out = out.view(out.size(0), -1)
        out = F.relu(self.fc1(out))
        out = F.relu(self.fc2(out))
        out = self.fc3(out)
        return out

In [11]:
Resnet_Shadow = torch.hub.load('pytorch/vision:v0.6.0', 'resnet18', pretrained=False)

Downloading: "https://github.com/pytorch/vision/archive/v0.6.0.zip" to /root/.cache/torch/hub/v0.6.0.zip


In [12]:
""" Target Model """

class Target_LeNet(nn.Module):
    def __init__(self):
        super(Target_LeNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1   = nn.Linear(16*5*5, 120)
        self.fc2   = nn.Linear(120, 84)
        self.fc3   = nn.Linear(84, 100)

    def forward(self, x):
        out = F.relu(self.conv1(x))
        out = F.max_pool2d(out, 2)
        out = F.relu(self.conv2(out))
        out = F.max_pool2d(out, 2)
        out = out.view(out.size(0), -1)
        out = F.relu(self.fc1(out))
        out = F.relu(self.fc2(out))
        out = self.fc3(out)
        return out

In [13]:
""" Attack Model Implementation """

class AttackModel(nn.Module):
  def __init__(self, input_size, hidden_size):
      super(AttackModel, self).__init__()
      self.input_size = input_size
      self.hidden_size  = hidden_size

      self.fc1 = torch.nn.Linear(self.input_size, self.hidden_size)
      self.fc2 = torch.nn.Linear(self.hidden_size, 2)
      # self.sigmoid = torch.nn.Softmax()

  def forward(self, x):
      x = self.fc1(x)
      x = F.softmax(self.fc2(x))
      output = x
      
      return output

In [14]:
def train_model(model, train_loader, test_loader, criterion, optimizer, scheduler, num_epochs=20, attack=False):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            
            if phase == 'train':
                model.train()  # Set model to training mode
                
                running_loss = 0.0
                running_corrects = 0
                
                for data in train_loader:
                    if attack:
                      inputs, labels = data
                    else:
                      inputs, labels, idx = data

                    inputs = inputs.to(device)
                    labels = labels.to(device)
                    
                    
                    optimizer.zero_grad()
                    with torch.set_grad_enabled(True):
                        outputs = model(inputs)
                        
                        _, preds = torch.max(outputs, 1)
                        # print(preds, labels)
                        loss = criterion(outputs, labels)
                        
                        loss.backward()
                        optimizer.step()
                    
                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)
                    
                    
                scheduler.step()
                print(running_corrects)
                epoch_loss = running_loss / train_size
                epoch_acc = running_corrects.double() / train_size

                print('{} Loss: {:.4f} Acc: {:.4f}'.format('Train', epoch_loss, epoch_acc))
                    
            else:
                
                model.eval()   # Set model to evaluate mode
                
                running_loss = 0.0
                running_corrects = 0
                
                for data in test_loader:
                    if attack:
                      inputs, labels = data
                    else:
                      inputs, labels, idx = data
                    
                    inputs = inputs.to(device)
                    labels = labels.to(device)
                    optimizer.zero_grad()
                    with torch.set_grad_enabled(False):
                        outputs = model(inputs)
                        _, preds = torch.max(outputs, 1)
                        
                        loss = criterion(outputs, labels)
                
                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)
                
                epoch_loss = running_loss / test_size
                epoch_acc = running_corrects.double() / test_size
                
                print('{} Loss: {:.4f} Acc: {:.4f}'.format('Val', epoch_loss, epoch_acc))
                if epoch_acc > best_acc:
                    best_acc = epoch_acc
                    best_model_wts = copy.deepcopy(model.state_dict())

                last_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    
    return model, best_acc, last_model_wts

In [15]:
""" Returns top3 prediction probability given x"""

def test(model, test_loader, criterion):
  model.eval()   # Set model to evaluate mode
                
  running_loss = 0.0
  running_corrects = 0
  # prediction_list = []
  i = 0
  for data in test_loader:
      inputs, labels, idx = data
      inputs = inputs.to(device)
      labels = labels.to(device)
     
      with torch.set_grad_enabled(False):
          outputs = model(inputs)
          _, preds = torch.max(outputs, 1)
          sm = torch.nn.Softmax()
          pred_probs = sm(outputs)
          pred_probs, indices = torch.sort(pred_probs, descending=True)
          # print(pred_probs)
          if i == 0:
            prediction_list = pred_probs[:,0:3]
          else:
            prediction_list = torch.cat((prediction_list, pred_probs[:,0:3]))
          i += 1
          loss = criterion(outputs, labels)

      running_loss += loss.item() * inputs.size(0)
      running_corrects += torch.sum(preds == labels.data)

  epoch_loss = running_loss / test_size
  epoch_acc = running_corrects.double() / test_size

  print('{} Loss: {:.4f} Acc: {:.4f}'.format('Val', epoch_loss, epoch_acc))
  
  return prediction_list

In [16]:
""" Attack model test function """
def attack_test(model, test_loader, criterion):
  model.eval()   # Set model to evaluate mode
                
  running_loss = 0.0
  running_corrects = 0
  # prediction_list = []
  i = 0
  test_true_label = []
  test_pred_label = []
  for data in test_loader:
      inputs, labels = data
      inputs = inputs.to(device)
      labels = labels.to(device)
      
      with torch.set_grad_enabled(False):
          outputs = model(inputs)
          _, preds = torch.max(outputs, 1)
          sm = torch.nn.Softmax()
          pred_probs = sm(outputs)
          pred_probs, indices = torch.sort(pred_probs, descending=True)
          loss = criterion(outputs, labels)

          test_true_label.append(labels.data)
          test_pred_label.append(preds.data)
      running_loss += loss.item() * inputs.size(0)
      running_corrects += torch.sum(preds == labels.data)

  epoch_loss = running_loss / test_size
  epoch_acc = running_corrects.double() / test_size

  print('{} Loss: {:.4f} Acc: {:.4f}'.format('Val', epoch_loss, epoch_acc))
  
  return test_true_label, test_pred_label

In [17]:
""" Train Shadow Model """

print("Training Shadow Model")
train_size = len(shadow_train)
test_size = len(shadow_test)
print(train_size, test_size)
shadow_net = Shadow_LeNet().to(device)
criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
# optimizer_ft = optim.Adam(shadow_net.parameters(), lr=0.001, weight_decay=1e-07)
optimizer_ft = optim.SGD(shadow_net.parameters(), lr=0.01, momentum=0.9, weight_decay=1e-07)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=20, gamma=0.1)

best_net, best_acc, last_net = train_model(shadow_net, shadow_train_loader, shadow_test_loader, criterion, optimizer_ft, exp_lr_scheduler,
                        num_epochs=60)

Training Shadow Model
12500 12500
Epoch 0/59
----------
tensor(185)
Train Loss: 4.5949 Acc: 0.0148
Val Loss: 4.5479 Acc: 0.0171

Epoch 1/59
----------
tensor(431)
Train Loss: 4.3632 Acc: 0.0345
Val Loss: 4.1941 Acc: 0.0471

Epoch 2/59
----------
tensor(861)
Train Loss: 4.0832 Acc: 0.0689
Val Loss: 4.0292 Acc: 0.0754

Epoch 3/59
----------
tensor(1157)
Train Loss: 3.9305 Acc: 0.0926
Val Loss: 3.9190 Acc: 0.1008

Epoch 4/59
----------
tensor(1405)
Train Loss: 3.8048 Acc: 0.1124
Val Loss: 3.8204 Acc: 0.1081

Epoch 5/59
----------
tensor(1684)
Train Loss: 3.6576 Acc: 0.1347
Val Loss: 3.7178 Acc: 0.1333

Epoch 6/59
----------
tensor(1950)
Train Loss: 3.5242 Acc: 0.1560
Val Loss: 3.6027 Acc: 0.1518

Epoch 7/59
----------
tensor(2244)
Train Loss: 3.4042 Acc: 0.1795
Val Loss: 3.5590 Acc: 0.1589

Epoch 8/59
----------
tensor(2387)
Train Loss: 3.2990 Acc: 0.1910
Val Loss: 3.5768 Acc: 0.1598

Epoch 9/59
----------
tensor(2674)
Train Loss: 3.2087 Acc: 0.2139
Val Loss: 3.4611 Acc: 0.1786

Epoch 10/

In [18]:
print("Training Base Shadow Model")
train_size = len(shadow_train)
test_size = len(shadow_test)
print(train_size, test_size)
base_shadow_net = Base_Shadow_LeNet().to(device)
criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
# base_optimizer_ft = optim.Adam(base_shadow_net.parameters(), lr=0.001, weight_decay=1e-07)
base_optimizer_ft = optim.SGD(base_shadow_net.parameters(), lr=0.01, momentum=0.9, weight_decay=1e-07)

# Decay LR by a factor of 0.1 every 7 epochs
base_exp_lr_scheduler = lr_scheduler.StepLR(base_optimizer_ft, step_size=20, gamma=0.1)

best_base_net, best_base_acc, last_base_net = train_model(base_shadow_net, shadow_train_loader, shadow_test_loader, criterion, base_optimizer_ft, base_exp_lr_scheduler,
                        num_epochs=60)

Training Base Shadow Model
12500 12500
Epoch 0/59
----------
tensor(191)
Train Loss: 4.5957 Acc: 0.0153
Val Loss: 4.5453 Acc: 0.0274

Epoch 1/59
----------
tensor(597)
Train Loss: 4.3452 Acc: 0.0478
Val Loss: 4.1805 Acc: 0.0647

Epoch 2/59
----------
tensor(1042)
Train Loss: 4.0128 Acc: 0.0834
Val Loss: 4.0005 Acc: 0.0855

Epoch 3/59
----------
tensor(1318)
Train Loss: 3.8380 Acc: 0.1054
Val Loss: 3.9313 Acc: 0.0930

Epoch 4/59
----------
tensor(1581)
Train Loss: 3.6982 Acc: 0.1265
Val Loss: 3.7442 Acc: 0.1335

Epoch 5/59
----------
tensor(1857)
Train Loss: 3.5589 Acc: 0.1486
Val Loss: 3.6722 Acc: 0.1350

Epoch 6/59
----------
tensor(2033)
Train Loss: 3.4640 Acc: 0.1626
Val Loss: 3.5956 Acc: 0.1494

Epoch 7/59
----------
tensor(2299)
Train Loss: 3.3542 Acc: 0.1839
Val Loss: 3.5452 Acc: 0.1662

Epoch 8/59
----------
tensor(2447)
Train Loss: 3.2475 Acc: 0.1958
Val Loss: 3.5047 Acc: 0.1681

Epoch 9/59
----------
tensor(2776)
Train Loss: 3.1418 Acc: 0.2221
Val Loss: 3.5411 Acc: 0.1689

Epo

In [19]:
""" Train ResNet Shadow """

print("Training ResNet Shadow")
train_size = len(shadow_train)
test_size = len(shadow_test)
print(train_size, test_size)
resnet_shadow = Resnet_Shadow.to(device)
# loading pretrained model
#     net = torch.load('LeNet_5_class_pretrained_model.tar')
criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
# resnet_optimizer_ft = optim.Adam(resnet_shadow.parameters(), lr=0.001, weight_decay=1e-07)
resnet_optimizer_ft = optim.SGD(resnet_shadow.parameters(), lr=0.01, momentum=0.9, weight_decay=1e-07)


# Decay LR by a factor of 0.1 every 7 epochs
resnet_exp_lr_scheduler = lr_scheduler.StepLR(resnet_optimizer_ft, step_size=20, gamma=0.1)

resnet_best_net, resnet_best_acc, resnet_last_net = train_model(resnet_shadow, shadow_train_loader, shadow_test_loader, criterion, resnet_optimizer_ft, resnet_exp_lr_scheduler,
                        num_epochs=25)

Training ResNet Shadow
12500 12500
Epoch 0/24
----------
tensor(963)
Train Loss: 4.3985 Acc: 0.0770
Val Loss: 3.9486 Acc: 0.1080

Epoch 1/24
----------
tensor(1958)
Train Loss: 3.5944 Acc: 0.1566
Val Loss: 3.6702 Acc: 0.1498

Epoch 2/24
----------
tensor(2681)
Train Loss: 3.2048 Acc: 0.2145
Val Loss: 3.4698 Acc: 0.1785

Epoch 3/24
----------
tensor(3422)
Train Loss: 2.8861 Acc: 0.2738
Val Loss: 3.3869 Acc: 0.2028

Epoch 4/24
----------
tensor(4189)
Train Loss: 2.5478 Acc: 0.3351
Val Loss: 3.3602 Acc: 0.2260

Epoch 5/24
----------
tensor(5067)
Train Loss: 2.2231 Acc: 0.4054
Val Loss: 3.4341 Acc: 0.2279

Epoch 6/24
----------
tensor(6141)
Train Loss: 1.8463 Acc: 0.4913
Val Loss: 3.6073 Acc: 0.2360

Epoch 7/24
----------
tensor(7222)
Train Loss: 1.4896 Acc: 0.5778
Val Loss: 3.7608 Acc: 0.2366

Epoch 8/24
----------
tensor(8221)
Train Loss: 1.1750 Acc: 0.6577
Val Loss: 4.0329 Acc: 0.2370

Epoch 9/24
----------
tensor(9173)
Train Loss: 0.8858 Acc: 0.7338
Val Loss: 4.3353 Acc: 0.2421

Epoch 

In [20]:
""" Training Target model """

target_train_loader = DataLoader(target_train, batch_size=64, shuffle=True, num_workers=0)
target_unk_loader = DataLoader(target_unknown, batch_size=64, shuffle=True, num_workers=0)

train_size = len(target_train)
test_size = len(target_unknown)
target_net = Target_LeNet().to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
# target_optimizer_ft = optim.Adamax(target_net.parameters(), lr=0.001, weight_decay=1e-07)
target_optimizer_ft = optim.SGD(target_net.parameters(), lr=0.01, momentum=0.9, weight_decay=1e-07)

# Decay LR by a factor of 0.1 every 7 epochs
target_exp_lr_scheduler = lr_scheduler.StepLR(target_optimizer_ft, step_size=30, gamma=0.1)

target_best_net, target_best_acc, target_last_net = train_model(target_net, target_train_loader, target_unk_loader, criterion, target_optimizer_ft, target_exp_lr_scheduler,
                        num_epochs=60)

Epoch 0/59
----------
tensor(175)
Train Loss: 4.5669 Acc: 0.0140
Val Loss: 4.4986 Acc: 0.0162

Epoch 1/59
----------
tensor(305)
Train Loss: 4.4569 Acc: 0.0244
Val Loss: 4.3973 Acc: 0.0285

Epoch 2/59
----------
tensor(568)
Train Loss: 4.2798 Acc: 0.0454
Val Loss: 4.1689 Acc: 0.0686

Epoch 3/59
----------
tensor(1029)
Train Loss: 4.0321 Acc: 0.0823
Val Loss: 3.9500 Acc: 0.0972

Epoch 4/59
----------
tensor(1372)
Train Loss: 3.8424 Acc: 0.1098
Val Loss: 3.7954 Acc: 0.1208

Epoch 5/59
----------
tensor(1694)
Train Loss: 3.6709 Acc: 0.1355
Val Loss: 3.7280 Acc: 0.1301

Epoch 6/59
----------
tensor(1903)
Train Loss: 3.5435 Acc: 0.1522
Val Loss: 3.6468 Acc: 0.1521

Epoch 7/59
----------
tensor(2176)
Train Loss: 3.4196 Acc: 0.1741
Val Loss: 3.5982 Acc: 0.1606

Epoch 8/59
----------
tensor(2416)
Train Loss: 3.3072 Acc: 0.1933
Val Loss: 3.5899 Acc: 0.1586

Epoch 9/59
----------
tensor(2588)
Train Loss: 3.2327 Acc: 0.2070
Val Loss: 3.5868 Acc: 0.1686

Epoch 10/59
----------
tensor(2834)
Train L

In [21]:
""" Generating Dataset (Train Set) for Attack Model using Shadow Model"""

print("Shadow model Test set predictions: ")
train_size = len(shadow_train)
test_size = len(shadow_test)
print(train_size, test_size)

s_net = Shadow_LeNet().to(device)
s_net.load_state_dict(last_net)

shadow_test_prediction = test(s_net, shadow_test_loader, criterion)
print("Shadow model Train set predictions: ")
shadow_train_prediction = test(s_net, shadow_train_loader, criterion)

d1 = torch.utils.data.TensorDataset(shadow_train_prediction, torch.ones(train_size, dtype=torch.long))

d2 = torch.utils.data.TensorDataset(shadow_test_prediction, torch.zeros(test_size, dtype=torch.long))

shadow_trained_dataset = torch.utils.data.ConcatDataset([d1, d2])

len(shadow_trained_dataset)

Shadow model Test set predictions: 
12500 12500


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Val Loss: 5.5835 Acc: 0.2014
Shadow model Train set predictions: 
Val Loss: 0.8464 Acc: 0.7888


25000

In [22]:
""" Generating Dataset (Base Train Set) for Attack Model using Base Shadow Model"""

train_size = len(shadow_train)
test_size = len(shadow_test)
print(train_size, test_size)

bs_net = Base_Shadow_LeNet().to(device)
bs_net.load_state_dict(last_base_net)
print("Shadow model Test set predictions: ")
base_shadow_test_prediction = test(bs_net, shadow_test_loader, criterion)
print("Shadow model Train set predictions: ")
base_shadow_train_prediction = test(bs_net, shadow_train_loader, criterion)

d1_base = torch.utils.data.TensorDataset(base_shadow_train_prediction, torch.ones(train_size, dtype=torch.long))

d2_base = torch.utils.data.TensorDataset(base_shadow_test_prediction, torch.zeros(test_size, dtype=torch.long))

base_shadow_trained_dataset = torch.utils.data.ConcatDataset([d1_base, d2_base])

len(base_shadow_trained_dataset)

12500 12500
Shadow model Test set predictions: 


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Val Loss: 6.3963 Acc: 0.1838
Shadow model Train set predictions: 
Val Loss: 0.6745 Acc: 0.8395


25000

In [23]:
print("Sample Base Shadow Trainset data: ", base_shadow_trained_dataset[17223][0], "Base Shadow Trainset label", base_shadow_trained_dataset[17223][1])

Sample Base Shadow Trainset data:  tensor([0.9378, 0.0272, 0.0093]) Base Shadow Trainset label tensor(0)


In [24]:
""" Generating Dataset (ResNet Train Set) for Attack Model using ResNet Shadow"""

train_size = len(shadow_train)
test_size = len(shadow_test)
print(train_size, test_size)
resnet_net = Resnet_Shadow.to(device)
resnet_net.load_state_dict(resnet_last_net)
print("Shadow model Test set predictions: ")
resnet_shadow_test_prediction = test(resnet_net, shadow_test_loader, criterion)
print("Shadow model Train set predictions: ")
resnet_shadow_train_prediction = test(resnet_net, shadow_train_loader, criterion)

d1_resnet = torch.utils.data.TensorDataset(resnet_shadow_train_prediction, torch.ones(train_size, dtype=torch.long))

d2_resnet = torch.utils.data.TensorDataset(resnet_shadow_test_prediction, torch.zeros(test_size, dtype=torch.long))

resnet_shadow_trained_dataset = torch.utils.data.ConcatDataset([d1_resnet, d2_resnet])

len(resnet_shadow_trained_dataset)

12500 12500
Shadow model Test set predictions: 


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Val Loss: 5.0208 Acc: 0.2815
Shadow model Train set predictions: 
Val Loss: 0.0011 Acc: 0.9998


25000

In [25]:
""" Generating Dataset (Validation Set) for Attack Model using Target Model"""

train_size = len(target_train)
test_size = len(target_unknown)

t_net = Target_LeNet().to(device)
t_net.load_state_dict(target_last_net)

target_test_prediction = test(t_net, target_unk_loader, criterion)
target_train_prediction = test(t_net, target_train_loader, criterion)

d1 = torch.utils.data.TensorDataset(target_train_prediction, torch.ones(train_size, dtype=torch.long))
d2 = torch.utils.data.TensorDataset(target_test_prediction, torch.zeros(test_size, dtype=torch.long))

target_trained_dataset = torch.utils.data.ConcatDataset([d1, d2])
len(target_trained_dataset)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Val Loss: 11.3953 Acc: 0.1740
Val Loss: 0.1043 Acc: 0.9875


25000

In [26]:
train_size = len(shadow_trained_dataset)
base_train_size = len(base_shadow_trained_dataset)
resnet_train_size = len(resnet_shadow_trained_dataset)

test_size = len(target_trained_dataset)
attack_model = AttackModel(3, 64).to(device)

atk_train_loader = DataLoader(shadow_trained_dataset, batch_size=10, shuffle=True, num_workers=0)
atk_base_train_loader = DataLoader(base_shadow_trained_dataset, batch_size=10, shuffle=True, num_workers=0)
atk_resnet_train_loader = DataLoader(resnet_shadow_trained_dataset, batch_size=10, shuffle=True, num_workers=0)


atk_test_loader = DataLoader(target_trained_dataset, batch_size=10, shuffle=True, num_workers=0)
optimizer_ft = optim.Adam(attack_model.parameters(), lr=0.01, weight_decay=1e-6)
base_optimizer_ft = optim.Adam(attack_model.parameters(), lr=0.01, weight_decay=1e-6)
resnet_optimizer_ft = optim.Adam(attack_model.parameters(), lr=0.01, weight_decay=1e-6)



In [27]:
attack_best_net, attack_best_acc, attack_last_net = train_model(attack_model, atk_train_loader, atk_test_loader, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=40, attack=True)


Epoch 0/39
----------


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  from ipykernel import kernelapp as app


tensor(14186)
Train Loss: 0.6827 Acc: 0.5674
Val Loss: 0.6450 Acc: 0.6589

Epoch 1/39
----------
tensor(14240)
Train Loss: 0.6822 Acc: 0.5696
Val Loss: 0.6558 Acc: 0.6318

Epoch 2/39
----------
tensor(14362)
Train Loss: 0.6809 Acc: 0.5745
Val Loss: 0.6519 Acc: 0.6508

Epoch 3/39
----------
tensor(14297)
Train Loss: 0.6804 Acc: 0.5719
Val Loss: 0.6307 Acc: 0.6832

Epoch 4/39
----------
tensor(14242)
Train Loss: 0.6811 Acc: 0.5697
Val Loss: 0.6354 Acc: 0.6827

Epoch 5/39
----------
tensor(14175)
Train Loss: 0.6807 Acc: 0.5670
Val Loss: 0.6519 Acc: 0.6358

Epoch 6/39
----------
tensor(14272)
Train Loss: 0.6810 Acc: 0.5709
Val Loss: 0.7035 Acc: 0.5000

Epoch 7/39
----------
tensor(14261)
Train Loss: 0.6807 Acc: 0.5704
Val Loss: 0.6546 Acc: 0.6327

Epoch 8/39
----------
tensor(14293)
Train Loss: 0.6808 Acc: 0.5717
Val Loss: 0.6417 Acc: 0.6616

Epoch 9/39
----------
tensor(14282)
Train Loss: 0.6808 Acc: 0.5713
Val Loss: 0.6490 Acc: 0.6501

Epoch 10/39
----------
tensor(14253)
Train Loss: 0.6

In [28]:
t_l, t_p = attack_test(attack_best_net, atk_test_loader, criterion)
l_true = []
l_pred = []
for d in t_l:
    l_list = d.cpu().numpy()
    for l in l_list:
        l_true.append(l)
# print(l_true)
for d in t_p:
    l_list = d.cpu().numpy()
    for l in l_list:
        l_pred.append(l)
# print(l_pred)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Val Loss: 0.6365 Acc: 0.6854


In [29]:
attack_base_best_net, attack_base_best_acc, attack_base_last_net = train_model(attack_model, atk_base_train_loader, atk_test_loader, criterion, base_optimizer_ft, base_exp_lr_scheduler, num_epochs=40, attack=True)

Epoch 0/39
----------


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  from ipykernel import kernelapp as app


tensor(14637)
Train Loss: 0.6748 Acc: 0.5855
Val Loss: 0.6388 Acc: 0.6657

Epoch 1/39
----------
tensor(14645)
Train Loss: 0.6746 Acc: 0.5858
Val Loss: 0.6567 Acc: 0.6265

Epoch 2/39
----------
tensor(14636)
Train Loss: 0.6744 Acc: 0.5854
Val Loss: 0.6383 Acc: 0.6735

Epoch 3/39
----------
tensor(14641)
Train Loss: 0.6745 Acc: 0.5856
Val Loss: 0.6233 Acc: 0.6828

Epoch 4/39
----------
tensor(14606)
Train Loss: 0.6743 Acc: 0.5842
Val Loss: 0.6338 Acc: 0.6716

Epoch 5/39
----------
tensor(14628)
Train Loss: 0.6741 Acc: 0.5851
Val Loss: 0.6246 Acc: 0.6734

Epoch 6/39
----------
tensor(14698)
Train Loss: 0.6741 Acc: 0.5879
Val Loss: 0.6398 Acc: 0.6548

Epoch 7/39
----------
tensor(14627)
Train Loss: 0.6748 Acc: 0.5851
Val Loss: 0.6348 Acc: 0.6713

Epoch 8/39
----------
tensor(14638)
Train Loss: 0.6746 Acc: 0.5855
Val Loss: 0.6264 Acc: 0.6766

Epoch 9/39
----------
tensor(14574)
Train Loss: 0.6745 Acc: 0.5830
Val Loss: 0.6274 Acc: 0.6812

Epoch 10/39
----------
tensor(14671)
Train Loss: 0.6

In [30]:
t_base_l, t_base_p = attack_test(attack_base_best_net, atk_test_loader, criterion)
l_base_true = []
l_base_pred = []
for d_base in t_base_l:
    l_base_list = d_base.cpu().numpy()
    for l_base in l_base_list:
        l_base_true.append(l_base)

for d_base in t_base_p:
    l_base_list = d_base.cpu().numpy()
    for l_base in l_base_list:
        l_base_pred.append(l_base)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Val Loss: 0.6331 Acc: 0.6901


In [31]:
attack_resnet_best_net, attack_resnet_best_acc, attack_resnet_last_net = train_model(attack_model, atk_resnet_train_loader, atk_test_loader, criterion, resnet_optimizer_ft, resnet_exp_lr_scheduler, num_epochs=40, attack=True)

Epoch 0/39
----------


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  from ipykernel import kernelapp as app


tensor(22755)
Train Loss: 0.4048 Acc: 0.9102
Val Loss: 0.7318 Acc: 0.5780

Epoch 1/39
----------
tensor(22971)
Train Loss: 0.3934 Acc: 0.9188
Val Loss: 0.7432 Acc: 0.5658

Epoch 2/39
----------
tensor(23024)
Train Loss: 0.3918 Acc: 0.9210
Val Loss: 0.7411 Acc: 0.5694

Epoch 3/39
----------
tensor(23059)
Train Loss: 0.3902 Acc: 0.9224
Val Loss: 0.7420 Acc: 0.5682

Epoch 4/39
----------
tensor(23098)
Train Loss: 0.3888 Acc: 0.9239
Val Loss: 0.7437 Acc: 0.5659

Epoch 5/39
----------
tensor(23123)
Train Loss: 0.3878 Acc: 0.9249
Val Loss: 0.7427 Acc: 0.5675

Epoch 6/39
----------
tensor(23115)
Train Loss: 0.3881 Acc: 0.9246
Val Loss: 0.7374 Acc: 0.5738

Epoch 7/39
----------
tensor(23157)
Train Loss: 0.3862 Acc: 0.9263
Val Loss: 0.7164 Acc: 0.5954

Epoch 8/39
----------
tensor(23143)
Train Loss: 0.3868 Acc: 0.9257
Val Loss: 0.7278 Acc: 0.5838

Epoch 9/39
----------
tensor(23143)
Train Loss: 0.3866 Acc: 0.9257
Val Loss: 0.7398 Acc: 0.5706

Epoch 10/39
----------
tensor(23186)
Train Loss: 0.3

In [32]:
t_resnet_l, t_resnet_p = attack_test(attack_resnet_best_net, atk_test_loader, criterion)
l_resnet_true = []
l_resnet_pred = []
for d_resnet in t_resnet_l:
    l_resnet_list = d_resnet.cpu().numpy()
    for l_resnet in l_resnet_list:
        l_resnet_true.append(l_resnet)

for d_resnet in t_resnet_p:
    l_resnet_list = d_resnet.cpu().numpy()
    for l_resnet in l_resnet_list:
        l_resnet_pred.append(l_resnet)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Val Loss: 0.7164 Acc: 0.5954


In [33]:
#Check Attack Accuracy using Shadow Model

zip_lists = zip(l_true, l_pred)
count = 0
for list_t, list_p in zip_lists:
  if list_t-list_p == 0:
    count += 1

accuracy = (count / len(l_true)) * 100

print("Attack Accuracy using Shadow Model: ", accuracy, "%")

Attack Accuracy using Shadow Model:  68.536 %


In [34]:
#Check Attack Accuracy using Base Shodow Model

zip_base_lists = zip(l_base_true, l_base_pred)
base_count = 0
for list_base_t, list_base_p in zip_base_lists:
  if list_base_t-list_base_p == 0:
    base_count += 1

base_accuracy = (base_count / len(l_base_true)) * 100

print("Attack Accuracy using Base Shadow Model: ", base_accuracy, "%")

Attack Accuracy using Base Shadow Model:  69.00800000000001 %


In [35]:
#Check Attack Accuracy using Base ResNet Shodow

zip_resnet_lists = zip(l_resnet_true, l_resnet_pred)
resnet_count = 0
for list_resnet_t, list_resnet_p in zip_resnet_lists:
  if list_resnet_t-list_resnet_p == 0:
    resnet_count += 1

resnet_accuracy = (resnet_count / len(l_resnet_true)) * 100

print("Attack Accuracy using ResNet Shadow: ", resnet_accuracy, "%")

Attack Accuracy using ResNet Shadow:  59.536 %
